In [44]:
import java.io.File

val input = File("9.txt").readLines()
    .map { line ->
        val parts = line.split(",").map { it.trim().toInt() }
        Pair(parts[0], parts[1])
    }

In [47]:
fun part1(coordinates: List<Pair<Int,Int>>): Long {
    var maxArea = 0L
    for (i in coordinates.indices) {
        for (j in i + 1 until coordinates.size) {
            val a = coordinates[i]
            val b = coordinates[j]
            val area = (abs(a.first - b.first) + 1L) * (abs(a.second - b.second) + 1L)
            maxArea = maxOf(maxArea, area)
        }
    }
    return maxArea
}
part1(input)


fun isPointInPolygon(point: Pair<Int, Int>, polygon: List<Pair<Int, Int>>): Boolean {
    var inside = false
    for (i in polygon.indices) {
        val p1 = polygon[i]
        val p2 = polygon[(i + 1) % polygon.size]
        if (p1.first == p2.first) {
            if (point.first == p1.first &&
                point.second in minOf(p1.second, p2.second)..maxOf(p1.second, p2.second)) {
                return true
            }
        } else if (p1.second == p2.second) {
            if (point.second == p1.second &&
                point.first in minOf(p1.first, p2.first)..maxOf(p1.first, p2.first)) {
                return true
            }
        }
        val intersect = ((p1.second > point.second) != (p2.second > point.second)) &&
                (point.first < (p2.first - p1.first) * (point.second - p1.second) /
                        (p2.second - p1.second).toDouble() + p1.first)
        if (intersect) {
            inside = !inside
        }
    }
    return inside
}

/**
 * Basically brute force, but we're gonna compress the coordinates so we don't have to check so many tiles
 *
 * */
fun part2(coordinates: List<Pair<Int, Int>>): Long {
    val perimeter = coordinates

    val allXs = coordinates.map { it.first }.distinct().sorted()
    val allYs = coordinates.map { it.second }.distinct().sorted()

    val xToIndex = allXs.mapIndexed { index, x -> x to index }.toMap()
    val yToIndex = allYs.mapIndexed { index, y -> y to index }.toMap()

    val grid = Array(allXs.size) { IntArray(allYs.size) { 0 } }

    for (xIdx in 0 until allXs.size - 1) {
        for (yIdx in 0 until allYs.size - 1) {
            val testX = (allXs[xIdx] + allXs[xIdx + 1]) / 2
            val testY = (allYs[yIdx] + allYs[yIdx + 1]) / 2

            var inside = false
            for (i in perimeter.indices) {
                val p1 = perimeter[i]
                val p2 = perimeter[(i + 1) % perimeter.size]

                if (p1.first == p2.first) {
                    if (testX == p1.first &&
                        testY in minOf(p1.second, p2.second)..maxOf(p1.second, p2.second)) {
                        inside = true
                        break
                    }
                } else if (p1.second == p2.second) {
                    if (testY == p1.second &&
                        testX in minOf(p1.first, p2.first)..maxOf(p1.first, p2.first)) {
                        inside = true
                        break
                    }
                }

                if (p1.second == p2.second) continue

                val intersect = ((p1.second > testY) != (p2.second > testY)) &&
                        (testX < (p2.first - p1.first).toDouble() * (testY - p1.second) /
                                (p2.second - p1.second).toDouble() + p1.first)

                if (intersect) inside = !inside
            }

            if (inside) {
                grid[xIdx][yIdx] = 1
            }
        }
    }

    val prefix = Array(allXs.size + 1) { IntArray(allYs.size + 1) { 0 } }
    for (x in 1 until allXs.size) {
        for (y in 1 until allYs.size) {
            prefix[x][y] = grid[x-1][y-1] + prefix[x-1][y] + prefix[x][y-1] - prefix[x-1][y-1]
        }
    }

    var maxArea = 0L

    for (i in coordinates.indices) {
        for (j in i + 1 until coordinates.size) {
            val a = coordinates[i]
            val b = coordinates[j]
            if (a.first == b.first || a.second == b.second) continue

            val minX = minOf(a.first, b.first)
            val maxX = maxOf(a.first, b.first)
            val minY = minOf(a.second, b.second)
            val maxY = maxOf(a.second, b.second)

            val potentialArea = (maxX - minX + 1L) * (maxY - minY + 1L)
            if (potentialArea <= maxArea) continue

            val x1Idx = xToIndex[minX] ?: continue
            val y1Idx = yToIndex[minY] ?: continue
            val x2Idx = xToIndex[maxX] ?: continue
            val y2Idx = yToIndex[maxY] ?: continue

            val minXIdx = minOf(x1Idx, x2Idx)
            val maxXIdx = maxOf(x1Idx, x2Idx)
            val minYIdx = minOf(y1Idx, y2Idx)
            val maxYIdx = maxOf(y1Idx, y2Idx)

            if (minXIdx == maxXIdx || minYIdx == maxYIdx) continue

            val sum = prefix[maxXIdx][maxYIdx] - prefix[minXIdx][maxYIdx] -
                    prefix[maxXIdx][minYIdx] + prefix[minXIdx][minYIdx]
            val expectedCells = (maxXIdx - minXIdx) * (maxYIdx - minYIdx)

            if (sum == expectedCells) {
                maxArea = potentialArea
            }
        }
    }

    return maxArea
}
part2(input)

1554370486